In [1]:
import pandas as pd
import yfinance as yf
import yahooquery as yq
import warnings
warnings.filterwarnings("ignore")


## reading csv file
companies = pd.read_csv("projectb.csv")
tickerList = companies["Ticker"]

# creating a list of dataframes for years 2020-2023 with the columns "wc/ta", "re/ta", "ebit/ta", "be/tl", "#shares", "price", "m2b"
dfs = []
columns = ["wc/ta", "re/ta", "ebit/ta", "be/tl", "#shares", "price", "m2b", "zScore"]
for i in range(4):
    dfs.append(pd.DataFrame(0, index=tickerList, columns=columns))
series = pd.Series(dfs)


#                Initialization Complete
#                Main Code begins


t = 0
for df in series:
    for ticker in tickerList:
        data = yq.Ticker(ticker)
        
        # extracting balance sheet values
        bs = data.balance_sheet()
        bs = bs[bs['TotalAssets']>0] # FirstFilter 
        bs.set_index('asOfDate', inplace = True)
        
        # extracting income statement values  
        it = data.income_statement()
        it = it[(it['periodType'] =='12M') & (it['TotalRevenue']>0)]
        it.set_index('asOfDate', inplace = True)
        
        # extracting cash flow statement values 
        cf = data.cash_flow()
        cf = cf[(cf['periodType'] =='12M') & (cf['EndCashPosition']>0)]
        cf.set_index('asOfDate', inplace = True)

        year = bs.index[t]
        
        # Computing the four elements of Modified Altman Z-Score

        # wc_ta
        ca = bs.loc[year, 'CurrentAssets']
        cl = bs.loc[year, 'CurrentLiabilities']
        ta = bs.loc[year, 'TotalAssets']
        wcTa = (ca-cl)/ta
        #print(ticker, ' Working Capital / Total Assets =', wcTa)
        df['wc/ta'][ticker] = wcTa

        #re_ta
        re = bs.loc[year, 'RetainedEarnings']
        reTa = re/ta
        #print(ticker,' Retained Earnings / Total Assets =', reTa)
        df['re/ta'][ticker] = reTa

        #ebit_ta
        ebit = it.loc[year, 'EBIT']
        ebitTa = ebit/ta
        #print(ticker,' EBIT / Total Assets =', ebitTa)
        df['ebit/ta'][ticker] = ebitTa

        #be_tl
        be = bs.loc[year, 'StockholdersEquity']
        tl = bs.loc[year, 'TotalLiabilitiesNetMinorityInterest']
        beTl = be/tl
        #print(ticker, ' Book Equity / Total Liabilities =', beTl)
        df['be/tl'][ticker] = beTl     

        #Computing M2B
        
        so = 0
        so = data.quotes[ticker]["sharesOutstanding"]
        price = yf.download(ticker,'2020-12-29','2023-12-31')['Adj Close']
        yearEnd = ['2020-12-31', '2021-12-31', '2022-12-30', '2023-12-29']
        
        sharePrice = price[yearEnd[t]]
        df['price'][ticker] = sharePrice
        df['#shares'][ticker] = so

        m2b = (so*sharePrice)/be
        df['m2b'][ticker] = m2b

    # Using Zscore Formula
    df["zScore"] = 3.25 + 6.56*df["wc/ta"] + 3.26*df["re/ta"] + 6.72*df["ebit/ta"] + 1.05*df["be/tl"]
    
    
    # Finding highest and lowest credit rating stocks 
    # Finding undervalued and overvalued stocks
    
    maxZscore = -9999
    minZscore = 9999
    maxM2b = -9999
    minM2b = 9999
    tickerLowM2b = ''
    tickerHighM2b = ''
    tickerLowZscore = ''
    tickerHighZscore = ''


    zScores = df["zScore"]
    m2bs = df["m2b"]

    for score in zScores:
        if score > maxZscore:
            maxZscore = score


        if score < minZscore:
            minZscore = score

    for m2b in m2bs:
        if m2b > maxM2b:
            maxM2b = m2b
        if m2b < minM2b:
            minM2b = m2b  

    #print(maxZscore)
    #print(minZscore)
    #print(maxM2b)
    #print(minM2b)
    
    
    # Computing investment into each stock for each year
    
    df["CreditInvestment"] = 0
    df["ValueInvestment"] = 0
    df["Portfolio"] = 0
    for ticker in tickerList:
        if df["zScore"][ticker] == maxZscore:
            tickerHighZscore = ticker
            #df["GoodCreditInvestment"] = 10000
        if df["zScore"][ticker] == minZscore:
            tickerLowZscore = ticker
            #df["GoodCreditInvestment"] = -10000
        if df["m2b"][ticker] == maxM2b:
            tickerHighM2b = ticker
            #df["UnderValuedInvestment"] = -10000
        if df["m2b"][ticker] == minM2b:
            tickerLowM2b = ticker
            #df["UnderValuedInvestment"] = 10000
    
    df["CreditInvestment"][tickerHighZscore] = 10000
    df["CreditInvestment"][tickerLowZscore] = -10000
    #if ticker == tickerHighM2b:
    df["ValueInvestment"][tickerHighM2b] = -10000
    #if ticker == tickerLowM2b:
    df["ValueInvestment"][tickerLowM2b] = 10000

    df["Portfolio"] = df["CreditInvestment"] + df["ValueInvestment"]
    #print("Good Credit:",tickerHighZscore," Bad Credit:",tickerLowZscore, " Over-valued:",tickerHighM2b," Under-valued:",tickerLowM2b, "\n")
    t+=1    
    

#Code works!

#Function that prints each year's P/L, Year Returns, Net P/L, Annualized Return

def computeValues():
    
    
    allReturns = []
    allPL = []
    #year = 0
    for i in range(3):

        #series[0]["price"]

        ##      Return
        yearRet = series[i+1]["price"]/series[i]["price"] - 1
        #year1Ret

        ##        P/L
        yearInvestment = series[i]["Portfolio"]
        yearPL = yearInvestment*yearRet
        #year1PL
        yearPLSum = yearPL.sum()
        totalYearInvestment = abs(yearInvestment).sum()

        ##      Portfolio Returns
        yearPortfolioReturn = (yearPLSum/totalYearInvestment)
        #yearPortfolioReturn

        # Append total P/L and portfolio returns to list
        allPL.append(yearPLSum)
        allReturns.append(yearPortfolioReturn)

    print("2021 P/L:",  allPL[0], "\n2021 Returns:", allReturns[0])
    print("\n2022 P/L:",  allPL[1], "\n2022 Returns:", allReturns[1])
    print("\n2023 P/L:",  allPL[2], "\n2023 Returns:", allReturns[2])


    netPL = allPL[0] + allPL[1] + allPL[2]
    retAnnualized = (((1+allReturns[0])*(1+allReturns[1])*(1+allReturns[2]))**(1/3) - 1)
    print("\nNet P/L:",netPL, "\nAnnualized Return:",retAnnualized,"or",retAnnualized*100,  "%")    

    
computeValues()








[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

2021 P/L: 4613.7443872686745 
2021 Returns: 0.23068721936343373

2022 P/L: 8143.425041113918 
2022 Returns: 0.40717125205569593

2023 P/L: -24296.190881930626 
2023 Returns: -0.6074047720482657

Net P/L: -11539.021453548034 
Annualized Return: -0.12068080498458333 or -12.068080498458333 %


In [3]:
import yfinance as yf
import yahooquery as yq
import pandas as pd


# Read the list of tickers from the csv file (assuming a CSV file with a column 'Ticker')
mydata = pd.read_csv('projectb.csv')
ticker = mydata['Ticker']


fin_data = yq.Ticker(ticker)
income_stmt = fin_data.income_statement()
income_stmt = income_stmt[(income_stmt['periodType'] == '12M') & (income_stmt['TotalRevenue'] > 0)]
income_stmt.set_index("asOfDate", inplace=True)
income_stmt.index.array[0] = '2020-12-31'
income_stmt.index.array[1] = '2021-12-31'
income_stmt.index.array[2] = '2022-12-31'
income_stmt.index.array[3] = '2023-12-31'


balance_sheet = fin_data.balance_sheet()
balance_sheet = balance_sheet[balance_sheet['TotalAssets'] > 0]
balance_sheet.set_index("asOfDate", inplace=True)
balance_sheet.index.array[0] = '2020-12-31'
balance_sheet.index.array[1] = '2021-12-31'
balance_sheet.index.array[2] = '2022-12-31'
balance_sheet.index.array[3] = '2023-12-31'

cash_flow = fin_data.cash_flow()

cash_flow.set_index('asOfDate', inplace=True)
cash_flow.index.array[0] = '2020-12-31'
cash_flow.index.array[1] = '2021-12-31'
cash_flow.index.array[2] = '2022-12-31'
cash_flow.index.array[3] = '2023-12-31'


# Calculating A = (Current Assets - Current Liabilities) / Total Assets
current_asset = balance_sheet['CurrentAssets']
current_liab = balance_sheet['CurrentLiabilities']
total_asset = balance_sheet['TotalAssets']
wc_ta = (current_asset - current_liab) / total_asset

# Calculating B = Retained Earnings / Total Assets
retain_earning = balance_sheet['RetainedEarnings']
re_ta = retain_earning / total_asset

# Calculating C = EBIT / Total Assets
ebit = income_stmt['EBIT']
ebit_ta = ebit / total_asset

#Calculating D = Book Equity / Total Liabilities
book_equity = balance_sheet['StockholdersEquity']
total_liabilities = balance_sheet['TotalLiabilitiesNetMinorityInterest']
be_tl = book_equity / total_liabilities


zscore = 3.25+ 6.56 * wc_ta + 3.26 * re_ta + 6.72 * ebit_ta + 1.05 * be_tl

print (zscore)


# Function to calculate M2B ratio

fin_data = yq.Ticker(ticker)

#so = fin_data.quotes[ticker]['sharesOutstanding']


#print(m2b_ratio)





asOfDate
2020-12-31     6.937732
2021-12-31     7.456286
2022-12-31     6.898660
2023-12-31     7.891602
2020-06-30     7.929642
2021-06-30     7.945155
2022-06-30     7.806283
2023-06-30     8.007342
2021-01-31    10.717602
2022-01-31    11.221787
2023-01-31     8.626257
2024-01-31    13.561120
2020-12-31     7.204080
2021-12-31     8.792099
2022-12-31    10.241200
2023-12-31     9.998555
dtype: float64


In [4]:
balance_sheet

,periodType,currencyCode,AccountsPayable,AccountsReceivable,AccumulatedDepreciation,AdditionalPaidInCapital,AllowanceForDoubtfulAccountsReceivable,AssetsHeldForSaleCurrent,AvailableForSaleSecurities,BuildingsAndImprovements,...,TotalEquityGrossMinorityInterest,TotalLiabilitiesNetMinorityInterest,TotalNonCurrentAssets,TotalNonCurrentLiabilitiesNetMinorityInterest,TotalTaxPayable,TradeandOtherPayablesNonCurrent,TreasurySharesNumber,TreasuryStock,WorkInProcess,WorkingCapital
asOfDate,,,,,,,,,,,,,,,,,,,,,
2020-12-31,12M,USD,9.505000e+09,1.357600e+10,-2.803800e+10,1.138900e+11,-293000000.0,0.000000e+00,NaN,1.250200e+10,...,6.327800e+10,1.116160e+11,1.236570e+11,6.912300e+10,1.392000e+09,6.559000e+09,4.873310e+08,3.849000e+10,2.040000e+09,8.744000e+09
2021-12-31,12M,USD,1.105500e+10,1.528300e+10,-2.871700e+10,1.230600e+11,-230000000.0,NaN,NaN,1.288200e+10,...,7.402300e+10,1.079950e+11,1.210390e+11,6.276900e+10,1.112000e+09,5.713000e+09,4.908780e+08,3.909900e+10,2.287000e+09,1.575300e+10
2022-12-31,12M,USD,9.889000e+09,1.403900e+10,-2.555200e+10,1.283450e+11,-169000000.0,5.830000e+09,NaN,1.147000e+10,...,7.680400e+10,1.105740e+11,1.320840e+11,5.477200e+10,2.220000e+09,4.306000e+09,5.062460e+08,4.169400e+10,1.577000e+09,-5.080000e+08
2023-12-31,12M,USD,9.632000e+09,1.487300e+10,-2.787800e+10,1.538430e+11,-166000000.0,0.000000e+00,NaN,1.237500e+10,...,6.877400e+10,9.878400e+10,1.140630e+11,5.250200e+10,2.993000e+09,2.881000e+09,7.127650e+08,7.566200e+10,1.952000e+09,7.213000e+09
2020-06-30,12M,USD,1.253000e+10,3.201100e+10,-4.319700e+10,NaN,-788000000.0,NaN,NaN,3.399500e+10,...,1.183040e+11,1.830070e+11,1.193960e+11,1.106970e+11,2.130000e+09,2.943200e+10,NaN,NaN,8.300000e+07,1.096050e+11
2021-06-30,12M,USD,1.516300e+10,3.804300e+10,-5.135100e+10,NaN,-751000000.0,NaN,NaN,4.392800e+10,...,1.419880e+11,1.917910e+11,1.493730e+11,1.031340e+11,2.174000e+09,2.719000e+10,NaN,NaN,7.900000e+07,9.574900e+10
2022-06-30,12M,USD,1.900000e+10,4.426100e+10,-5.966000e+10,NaN,-633000000.0,NaN,NaN,5.501400e+10,...,1.665420e+11,1.982980e+11,1.951560e+11,1.032160e+11,4.067000e+09,2.606900e+10,NaN,NaN,8.200000e+07,7.460200e+10
2023-06-30,12M,USD,1.809500e+10,4.868800e+10,-6.825100e+10,NaN,-650000000.0,NaN,NaN,6.846500e+10,...,2.062230e+11,2.057530e+11,2.277190e+11,1.016040e+11,4.152000e+09,2.556000e+10,NaN,NaN,2.300000e+07,8.010800e+10
2021-01-31,12M,USD,1.149000e+09,2.429000e+09,-1.408000e+09,8.719000e+09,-21000000.0,NaN,NaN,7.960000e+08,...,1.689300e+10,1.189800e+10,1.273600e+10,7.973000e+09,6.100000e+07,8.360000e+08,1.380000e+10,1.075600e+10,4.570000e+08,1.213000e+10
